# PSB and ESR

Combination of `august/ESR.ipynb`, `august/pauli.ipynb` and `august/pauli_plots.ipynb`

In [1]:
%matplotlib inline
import local_broom

Added 'quench/' and 'quench/libraries' for easy importing.


In [2]:
# Automagically reload modules when they are edited
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
from qcodes import Instrument, Station
from monty import Monty
import MDAC
from may.dots import getvoltages
from may.custom_devices import connect_to_gb, newSiDot
from liveplot import LivePlot
from fridge import Fridge
import paul
from shfqc import SHFQC
from monty import Monty
import time
import logging
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

## Connect

In [ ]:
channels = ["ST", "P1", "P2", "J"]
channel_addr = [0, 1, 2, 3]

#channels = ["P1", "P2", "J", "MW"]
#channel_addr = [1, 2, 3, 5]

# channels = ["MW"]
# channel_addr = [5]

shfqc = SHFQC(channels, channel_addr, "measure")
shfqc.connect()

In [ ]:
try:
    mdac = Instrument.find_instrument("mdac")
    mdac.close()
except KeyError:
    print('Attempting to remove instrument with name mdac. Does not exist')

mdac = MDAC.MDAC('mdac', 'ASRL11::INSTR')

try:
    lockin = Instrument.find_instrument("lockin")
    lockin.close()
except KeyError:
    print("Cannot remove instrument with name lockin. Does not exist")

scfg = Station(config_file='/Users/LD2007/Documents/Si_CMOS_james/measurements/system.yaml')

#lockin = scfg.load_instrument('sr860_top')
#psg = scfg.load_instrument('PSG')

lockin = SR860('lockin', 'TCPIP::192.168.0.108::inst0::INSTR')
psg = E8267('psg', 'TCPIP0::192.168.0.109::inst0::INSTR')


gb_control_si = connect_to_gb(mdac)  # Create our custom MDAC mappings
si = newSiDot(mdac)
fridge = Fridge("BlueFors_LD")

Disconnect. Run with caution

In [ ]:
# Disconnect from all devices
disconnect = False
if disconnect:
    shfqc.disconnect()
    try:
        mdac = Instrument.find_instrument("mdac")
        mdac.close()
    except KeyError:
        print('Attempting to remove instrument with name mdac. Does not exist')
    try:
        lockin = Instrument.find_instrument("lockin")
        lockin.close()
    except KeyError:
        print("Cannot remove instrument with name sr860_top. Does not exist")
    try:
        psg = Instrument.find_instrument("psg")
        psg.close()
    except KeyError:
        print('Attempting to remove instrument with name mdac. Does not exist')

In [ ]:
# setup monty
experiment = {
    "desc": "Run ESR frequency sweeps and PSB detuning / J sweeps"
}

monty = Monty("rf.esr_and_psb", experiment)

## Define parameters

> TODO: Convert naming scheme from "amplitude_volts" to "sweep_detuning"

In [ ]:
params = {
    "amplitude_volts": {  # define the amplitudes that should be used. (maximum 340mV, give in Volts)
        "mixed_pulse": {  # voltage amplitude for mixed state preperation
            "P1": -0.2,  # make sure keys match drive lines above
            "P2": 0.2,
        },
        "measure_pulse_start": {  # FOR PSB ONLY
            "P1": 0.0,  # voltages for stepping over when sweeping
            "P2": 0.0,
            "J": 0,
        },
        "measure_pulse_end": {  # PSB ONLY
            "P1": -0.006,
            "P2": 0.006,
            "J": 0,
        },
        "readout_pulse": {  # FOR ESR ONLY
            "P1": -0.0021,  #-0.0025,
            "P2": 0.0021,  #0.0025,
            "J": 0.0,
        },
        "ramp_rate": 13e-5,  # V/sample
    },
    "sweep_j": {  # FOR ESR ONLY
        "j_start": -0.15,
        "j_stop": -0.15,
    },
    "sweep_detuning": {  # FOR ESR ONLY
        "p1_start": -0.0015,
        "p1_stop": -0.0035,
        "p2_start": 0.0015,
        "p2_stop": 0.0035,
    },
    "readout": {
        "freq": 406.6e6,  # (Hz) try 406.2e6
        "gain": 0.95,  # If we set this to 1, then output overloads
        "time": 100e-6,  # sec
    },
    "mw": {  # Microwave ESR settings
        "freqs": {  # (Hz)
            "center": 1.0e9,  # 1.5e9,
            "start": 0, #  -400e6,
            "stop": 0, #  400e6,
            # steps determined by averaging/num_freqs
        },
        "psgfreqstart": 6.75e9,  # using upper sideband
        "psgfreqstop": 7.25e9,  # using upper sideband
        "gain": 0.8,  # If we set this to 1, then output overloads
        "span": 1e6,  # frequency span for chirp signal, if enabled
        "sampling_divider": 3,
    },
    "feedback": {  # FOR PSB ONLY
        "target": -24.1,  # in dB
        "stepsize": 2e-2,  # arbitrary units
        "slope": -0,  # -1, 1 depending on slope, 0 to disable feedback
    },
    "timings_sec": {  # all in seconds
        "mixed_initilise": 600e-6,
        "read": 100e-6,
        "settle": 3e-3,
        "trigger": 8e-3,  # metronome of the internal trigger
        "buffer": 10e-6,
        "mw_pulse": 50e-6,
        "sampling_divider": 6,  # playback waveform resolution 2^X
    },
    "powers": {  # in dB presets from UI
        "acq_in": 10,
        "acq_out": -10,
        "drive": 0,  # only 0 is supported
        "mw_drive": 0,  # conncted to to PSG
        "psg_drive": 7,  # just a guess for now, output of psg
    },
    "averaging": {
        "seqc_averages": 200,  # max 2**19 points
        "num_freqs": 501,
        "num_j": 1,
        "num_detuning": 50,
    },
    "gates": getvoltages(mdac),
    "temp": f"Mixing chamber {fridge.temp()} K",
}


# Dynamic variables
# create arrays of our sweeping paramters
# freq_steps = np.linspace(params["mw"]["freqs"]["start"],
#                     params["mw"]["freqs"]["stop"],
#                     params["averaging"]["num_freqs"])

freq_steps = np.linspace(params["mw"]["psgfreqstart"],
                    params["mw"]["psgfreqstop"],
                    params["averaging"]["num_freqs"])
j_steps = np.linspace(params["sweep_j"]["j_start"],
                    params["sweep_j"]["j_stop"],
                    params["averaging"]["num_j"])
p1_steps = np.linspace(params["sweep_detuning"]["p1_start"],
                    params["sweep_detuning"]["p1_stop"],
                    params["averaging"]["num_detuning"])
p2_steps = np.linspace(params["sweep_detuning"]["p2_start"],
                    params["sweep_detuning"]["p2_stop"],
                    params["averaging"]["num_detuning"])